In [1]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
from base_layer_utils import compute_layer1_oof
from base_layer_utils import SklearnBLE

#from fast_text_data import FastTextDataGenerator# fasttext_data_process 
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import time
import gc
import xgboost as xgb
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('~/data/toxic/data/train_preprocessed_clean.csv')
train_ori = pd.read_csv('~/data/toxic/data/train.csv')
train = train.merge(train_ori[['comment_text', 'id']], on='id', suffixes=('', '_ori'))
train.head(2)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity,comment_text_polarity,comment_text_ori
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0,explanation why the edits made under my userna...,Explanation\nWhy the edits made under my usern...
1,d aww he matches this background colour i m se...,000103f0d9cfb60f,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0,d aww he matches this background colour i m se...,D'aww! He matches this background colour I'm s...


In [3]:
test = pd.read_csv('~/data/toxic/data/test_preprocessed_clean.csv')
test_ori = pd.read_csv('~/data/toxic/data/test.csv')
test = test.merge(test_ori[['comment_text', 'id']], on='id', suffixes=('', '_ori'))
test.head(2)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity,comment_text_polarity,comment_text_ori
0,yo bitch ja rule is more succesful then you ll...,00001cee341fdb12,NaN,NaN,NaN,test,NaN,NaN,NaN,NaN,yo bitch ja rule is more succesful then you ll...,Yo bitch Ja Rule is more succesful then you'll...
1,from rfc the title is fine as it is imo,0000247867823ef7,NaN,NaN,NaN,test,NaN,NaN,NaN,NaN,from rfc the title is fine as it is imo cleane...,== From RfC == \n\n The title is fine as it is...


train = train.head(10000)
test = test.head(10000)

In [4]:
train.shape, test.shape

((159571, 12), (153164, 12))

In [5]:
PATH = '~/data/toxic/data/'
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [6]:
test.drop(label_cols, axis=1, inplace=True)
test.shape

(153164, 6)

# tfidf data

In [7]:
bldr = BaseLayerDataRepo()

In [8]:
tfidfdata_compatible_models= [ModelName.LGB, ModelName.LGB_PERLABEL, ModelName.NBLGB, ModelName.NBLGB_PERLABEL,
                              ModelName.XGB, ModelName.XGB_PERLABEL, ModelName.NBXGB, ModelName.NBXGB_PERLABEL,
                              ModelName.LOGREG, ModelName.LOGREG_PERLABEL, ModelName.NBLOGREG, ModelName.NBLOGREG_PERLABEL,
                              ModelName.LSVC, ModelName.LSVC_PERLABEL, ModelName.NBLSVC, ModelName.NBLSVC_PERLABEL]

In [9]:
bldr.add_tfidf_data(train_sentence=train['comment_text'], test_sentence=test['comment_text'], 
                    y_train=train[label_cols], label_cols=label_cols, 
                    compatible_models=tfidfdata_compatible_models, word_ngram=(1,1), word_max=30000)

fitting word
transforming train word
transforming test word
tfidf(word level) done
tfidf_word_(1, 1)_30000_1_1.0 is added to the base layer data repo


In [10]:
print(bldr)

data_id: tfidf_word_(1, 1)_30000_1_1.0 
	x_train: (159571, 30000)	x_test: (153164, 30000)
	y_train type: <class 'dict'>
	compatible_models: {<ModelName.NBXGB: 2>, <ModelName.NBLGB: 6>, <ModelName.XGB: 1>, <ModelName.NBXGB_PERLABEL: 4>, <ModelName.LOGREG: 9>, <ModelName.LOGREG_PERLABEL: 11>, <ModelName.NBLOGREG: 10>, <ModelName.NBLSVC_PERLABEL: 16>, <ModelName.NBLGB_PERLABEL: 8>, <ModelName.NBLSVC: 14>, <ModelName.LSVC_PERLABEL: 15>, <ModelName.XGB_PERLABEL: 3>, <ModelName.NBLOGREG_PERLABEL: 12>, <ModelName.LSVC: 13>, <ModelName.LGB_PERLABEL: 7>, <ModelName.LGB: 5>}
 


# Start generating oof_train, oof_test,  layer1 estimater prediction and model data id list

In [12]:
# uncomment any model to add to the model pool

model_pool = {}

SEED = 1001

from sklearn.linear_model import LogisticRegression
###################################  Logreg normal   ####################################
logreg_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED)
model_pool[ModelName.LOGREG] = logreg_ble
###################################  Logreg nb       ####################################
nblogreg_params = {'C':0.25}
nblogreg_ble = SklearnBLE(LogisticRegression, nb=True, seed=SEED, params=nblogreg_params)
model_pool[ModelName.NBLOGREG] = nblogreg_ble
###################################  Logreg per label  ##################################
logreg_params_per_label = {
    'identity_hate': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
    'insult': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
    'obscene': {'C': 0.7, 'class_weight': 'balanced', 'fit_intercept': True},
    'severe_toxic': {'C': 0.3,'class_weight': None, 'fit_intercept': True},
    'threat': {'C': 0.05, 'class_weight': 'balanced', 'fit_intercept': True}, 
    'toxic': {'C': 0.8, 'class_weight': 'balanced', 'fit_intercept': True}
}
logreg_per_label_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED, per_label_params=logreg_params_per_label)
model_pool[ModelName.LOGREG_PERLABEL] = logreg_per_label_ble



# from sklearn.svm import LinearSVC
# ###################################  Logreg normal   ####################################
# lsvc_ble = SklearnBLE(LinearSVC, need_calibrated_classifier_cv=True)
# model_pool[ModelName.LSVC] = lsvc_ble
# ###################################  Logreg nb   ####################################
# nblsvc_params = {'C':0.02}
# nblsvc_ble = SklearnBLE(LinearSVC, nb=True, seed=SEED, params=nblogreg_params, need_calibrated_classifier_cv=True)
# model_pool[ModelName.NBLSVC] = nblsvc_ble


# from xgboost import XGBClassifier
# ###################################  XGB normal   ####################################
# xgb_params = {'n_jobs': 10}
# xgb_ble = SklearnBLE(XGBClassifier, seed=SEED, params=xgb_params)# XGBoostBLE(params=xgb_params, nb=False, seed=SEED)
# model_pool[ModelName.XGB] = xgb_ble
# ###################################  XGB nb   ####################################
# nbxgb_ble = SklearnBLE(XGBClassifier, nb=True, seed=SEED, params=xgb_params)
# model_pool[ModelName.NBXGB] = nbxgb_ble



# from lightgbm import LGBMClassifier
# ###################################  LGB normal   ####################################
# lgb_params = {'n_jobs': 8,'n_estimators': 50}
# lgb_ble = SklearnBLE(LGBMClassifier, seed=SEED, params=lgb_params)
# model_pool[ModelName.LGB] = lgb_ble
# ###################################  LGB nb   ####################################
# nblgb_ble = SklearnBLE(LGBMClassifier, nb=True, seed=SEED, params=lgb_params)
# model_pool[ModelName.NBLGB] = nblgb_ble
# ###################################  LGB per label   ####################################
# lgb_params_per_label = {}
# lgb_params_per_label['toxic'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['severe_toxic'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['obscene'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['threat'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['insult'] = {'n_jobs': 8,'num_leaves': 11}
# lgb_params_per_label['identity_hate'] = {'n_jobs': 8, 'num_leaves': 61}
# nblgb_per_label_ble = SklearnBLE(LGBMClassifier, seed=SEED, per_label_params=lgb_params_per_label)
# model_pool[ModelName.NBLGB_PERLABEL] = nblgb_per_label_ble

In [13]:
list(model_pool)

[<ModelName.LOGREG: 9>,
 <ModelName.NBLOGREG: 10>,
 <ModelName.LOGREG_PERLABEL: 11>]

In [14]:
type(train.head(2)['id'].values)

numpy.ndarray

In [15]:
layer1_est_preds, layer1_oof_train, layer1_oof_mean_test, model_data_id_list = compute_layer1_oof(bldr, model_pool, label_cols, sfm_threshold='median')

Computing... label: toxic        model_data_id: ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
dimension before selecting: train:(159571, 30000) test:(153164, 30000)
dimension after selecting: train:(159571, 15000) test:(153164, 15000)
Computing... label: toxic        model_data_id: ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0
dimension before selecting: train:(159571, 30000) test:(153164, 30000)
dimension after selecting: train:(159571, 15000) test:(153164, 15000)
Computing... label: toxic        model_data_id: ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0
dimension before selecting: train:(159571, 30000) test:(153164, 30000)
dimension after selecting: train:(159571, 15000) test:(153164, 15000)
Computing... label: severe_toxic model_data_id: ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
dimension before selecting: train:(159571, 30000) test:(153164, 30000)
dimension after selecting: train:(159571, 15000) test:(153164, 15000)
Computing... label: severe_toxic model_data_i

### submit the layer 1 estimator predictions. If they look fine, save them to BaseLayerResultsRepo

In [16]:
list(layer1_est_preds)

['ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0']

### generate files to submit from layer 1 estimators

In [22]:
def write_predictions_to_file(base_layer_est_preds):
    for key in base_layer_est_preds:
        submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
        submission[label_cols] = base_layer_est_preds[key]
        sub_id = int(time.time())
        print(sub_id)
        submission.to_csv('./BaseEstPreds/' + key + '_' + str(sub_id) + '.csv.gz', index=False, compression='gzip')

In [23]:
write_predictions_to_file(layer1_est_preds)

1522094161
1522094166
1522094171


# sanity check before save to BaseLayerResultsRepo

In [25]:
model_data_id_list # model_data we just computed in the layer 1

['ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0']

In [27]:
list(layer1_oof_train) # list keys (which are labels)

['obscene', 'identity_hate', 'severe_toxic', 'threat', 'insult', 'toxic']

In [28]:
len(layer1_oof_train['toxic']) # number of models to stack (each model will predict one set of toxic, servere_toxic, etc..)

3

In [29]:
len(layer1_oof_train['toxic'][0]) # examples in oof_train (meta features, x_train) (meta labels are in train[label])

159571

In [31]:
list(layer1_oof_mean_test)

['obscene', 'identity_hate', 'severe_toxic', 'threat', 'insult', 'toxic']

In [32]:
len(layer1_oof_mean_test['toxic'][0]) # examples in oof_test (will be used by meta model (after validation) to predict the final prediction)

153164

In [33]:
# read the doc string to BaseLayerResultsRepo to set params
base_layer_results_repo = BaseLayerResultsRepo(load_from_file=False, filepath='obj/WithPreprocessedFile/')

In [34]:
base_layer_results_repo.add(layer1_oof_train, layer1_oof_mean_test, layer1_est_preds, model_data_id_list)

In [35]:
scores = base_layer_results_repo.show_scores()

0	ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0
0	ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
0	ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0


In [36]:
 # let's give some fake scores
base_layer_results_repo.add_score('ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0', 0.9888)
base_layer_results_repo.add_score('ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0', 0.9777)
base_layer_results_repo.add_score('ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0', 0.9666)

ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9888
ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9777
ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9666


In [37]:
scores = base_layer_results_repo.show_scores()

0.9888	ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0
0.9777	ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
0.9666	ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0


In [38]:
base_layer_results_repo.save()

In [17]:
# Next step: go to stack_layer2

In [18]:
print('stack 1')

stack 1
